In [3]:
# Import pandas using Python 3.11.9 interpreter
import pandas as pd
import yfinance as yf
import datetime as dt


In [15]:
assets = ['NVDA', 'GME', 'TSLA', 'ACLS']
benchmarks = ['spy']

benchmarks = [asset.upper() for asset in benchmarks]
assets = [asset.upper() for asset in assets]

tickers = assets + benchmarks

startdate = '2019-09-18'


In [19]:
data = yf.download(tickers, start = startdate)['Adj Close']
assets = []
for asset in assets:
    assets2.append(asset.upper())

[*********************100%***********************]  5 of 5 completed
